##  Statement of Purpose
Our company builds Andriod and iOS apps that are available on Google Play and the App Store. We build apps that are free to download and install. This means our revenue is influenced by the number of users who use our app and see and engage in our in-app ads. As a result, the more users that see the ads, the better. 

The goal for this project is to analyze data to help our developers understand which type of app is likely to attract more users.

# Functions

In [77]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
   

def detect_English(string):
    foreignCount = 0
    for character in string:
        
        if ord(character) > 127:
            foreignCount += 1
    
    if foreignCount > 3:
        return False
        
    return True


def freq_table(dataset,index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
        
    table_percentages = {}
    for key in table:
        table_percentages[key] = (table[key]/total) * 100
    
    return table_percentages


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        

## Access and Explore the Data

In [2]:
from csv import reader

openfile = open("googleplaystore.csv",encoding='utf8')
read_file = reader(openfile)
GooglePlay = list(read_file)
GooglePlay_Headers = GooglePlay[0]
GooglePlay = GooglePlay[1:]


openfile = open("AppleStore.csv",encoding="utf8")
read_file = reader(openfile)
AppleStore = list(read_file)
AppleStore_Headers = AppleStore[0]
AppleStore = AppleStore[1:]

print(GooglePlay_Headers)
print('\n')
print(explore_data(GooglePlay,0,3,rows_and_columns=True))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13
None


The Google Play dataset has 10841 apps and 13 columns. Upon further inspection of the columns, the ones that will be of most use to us will be "App", "Category", "Rating", "Reviews", "Installs","Price", and "Genres".

Now let's explore the Apple Store dataset

In [3]:
print(AppleStore_Headers)
print("\n")
explore_data(AppleStore,0,3,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


The Apple Store dataset has 7197 apps and 16 columns. Upon further inspection of the columns, the ones that will be of most use to us will be "track_name", "currency", "price", "rating_count_tot", "rating_count_ver", "user_rating", "user_rating_ver", and "prime_genre".


## Cleaning the Data

Becaues we only build apps that are *free* to download and install, and focus only on English-speaking audiences, we need to remove non-English apps and apps that are not free, as well as any incorrect entries (an entry who is missing a column value)

The Google Play data set has a dedicated discussion section, and we can see that one of the discussions outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [4]:
print(GooglePlay[10472])  # incorrect row
print('\n')
print(GooglePlay_Headers)  # header
print('\n')
print(GooglePlay[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. As a consequence, we'll delete this row.

In [5]:
del GooglePlay[10472]

In [11]:
len(GooglePlay)

10840

### Removing Duplicate Entries

Upon further inspection of the Google Play data set, duplicate entries will be discovered. For example, Instagram has 4 different entries.

In [6]:
for app in GooglePlay:
    name = app[0]
    if name == "Instagram":
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


After searching for duplicate entries, 1,181 cases are discovered:

In [7]:
unique_entry = []
duplicate_entry = []

for app in GooglePlay:
    name = app[0]
    if name in unique_entry:
        duplicate_entry.append(name)
    else:
        unique_entry.append(name)
        
print("Number of duplicate apps: ",len(duplicate_entry))
print('\n')
print('Examples of duplicate apps:', duplicate_entry[:10])

Number of duplicate apps:  1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


We want to retain only one entry for each duplicate app in this dataset. However, we want to retain the entry that is the most up to date.

Looking back at the Instagram example, the main difference between the entries is the 4th position for each entry, which corresponds to the number of reviews. The difference in this value shows that these entries occur at different point in times. The lowest number of reviews correlate to the oldest entry, and the highest number of reviews correlate to the newst entry. Therefore we want to retain only the entry that has the highest number of reviews.

To do that, we will:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

In [12]:
app_views = {}

for application in GooglePlay:
    name = application[0]
    numberOfReviews = float(application[3])
    
    if name in app_views and app_views[name] < numberOfReviews:
        app_views[name]= numberOfReviews
        
    elif name not in app_views:
        app_views[name] = numberOfReviews    

Previously we identified 1,181 duplicate cases, so the length of the dictionary should be equal to the difference between the length of our data set and 1,181

In [19]:
print("Expected length:", len(GooglePlay)- 1181)
print("Actual length:",len(app_views))

Expected length: 9659
Actual length: 9659


Now that we have identified the unique values we want to keep, we will use the app_views dictionary to remove duplicates and keep only the entries with the highest number of views.

First create 2 empty lists, GooglePlay_clean and already_in_clean. Iterate through the GooglePlay dataset, and we add the row to the GooglePlay_clean list and the app name to the already_in_clean list if:
- The number of Reviews of the current app matches the number of reviews of that app as described in app_review dictionary.
- The name of the app is not already in already_in_clean. We need this to account for the duplicates that are not related to different number of Reviews.

In [16]:
GooglePlay_clean = []
already_in_clean = []

for row in GooglePlay:
    name = row[0]
    numberofReviews = float(row[3])
    
    if (app_views[name] == numberofReviews) and (name not in already_in_clean):
        GooglePlay_clean.append(row)
        already_in_clean.append(name)
            

Using the explore_data function, we confirm that we have 9,659 rows.

In [18]:
explore_data(GooglePlay_clean,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


## Removing Non-English Applications

If you explore both datasets, you will find that the naes of some of the apps suggest they are not intended for an English-speaking audience. Here are a few examples:

In [25]:
print(GooglePlay_clean[4412][0])
print(GooglePlay_clean[7940][0])
print('\n')
print(AppleStore[813][1])
print(AppleStore[6731][1])

中国語 AQリスニング
لعبة تقدر تربح DZ


爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


A new function detect_English was created to return False for non-English app titles. However, because we want to include symbols and emojis if they are present in the title, this function will return False only if the title has more than three characters that fall outside the ASCII range 0-127 (the range for English characters). This function is not perfect but should be fairly effective. Lets use the function to filter out non-English apps.

In [71]:
GooglePlay_clean2 = []
Google_NonEnglish = []

for row in GooglePlay_clean:
    name = row[0]
    
    if detect_English(name) == False:
        Google_NonEnglish.append(row)
    elif detect_English(name) == True:
        GooglePlay_clean2.append(row)

AppleStore_Clean = []
Apple_nonEnglish = []

for row in AppleStore:
    name = row[1]
    
    if detect_English(name) == False:
        Apple_nonEnglish.append(row)
    elif detect_English(name) == True:
        AppleStore_Clean.append(row)

After filtering out Non-English named applications, we identified 45 apps that were removed from the Google Play dataset and 1014 apps from the Apple Store dataset.

We are left with 9,614 applications in Google Play and 6,183 in the Apple Store dataset

In [72]:
print("Google Play Store Number of Non-English Apps:", len(Google_NonEnglish))
print("Google Play Store Number of English Apps:",len(GooglePlay_clean2))
print('\n')
print('Apple Store Number of Non-English Apps',len(Apple_nonEnglish))
print("Apple Store Number of English Apps", len(AppleStore_Clean))

Google Play Store Number of Non-English Apps: 45
Google Play Store Number of English Apps: 9614


Apple Store Number of Non-English Apps 1014
Apple Store Number of English Apps 6183


# Isolating the Free Applications

As mentioned previously, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Both data sets contain both free and non-free apps, so we will isolate the free apps before conducting our analysis. 

In [76]:
GooglePlay_Free = []
AppleStore_Free = []

for row in GooglePlay_clean2:
    IsFree = row[6]
    if IsFree == "Free":
        GooglePlay_Free.append(row)

for row in AppleStore_Clean:
    IsFree = float(row[4])
    if IsFree == 0.0:
        AppleStore_Free.append(row)
        
print("Google Play Store Number of Free Apps:", len(GooglePlay_Free))
print('\n')
print('Apple Store Number of Free Apps',len(AppleStore_Free))

Google Play Store Number of Free Apps: 8863


Apple Store Number of Free Apps 3222


After isolating the free apps, we have 8,863 applications remaining in Google Play and 3,222 apps remaining in the Apple Store datasets.

# Most Common Apps by Genre

As mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people who use our apps. 

To minimize risks and overhead costs, our validation strategy for an app idea is comprised of 3 steps:
1. Build a minimal Android version of the app, and add it to Google Play
2. If the app has a good response from users, develop it further
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store

We need to find app profiles that are successful on both markets. For instance, a productivity app might work well for both markets. Let's start by understanding the most common genres for each market. For this, we will build a Frequency table for the prime_genre column of the AppleStore data set, and the Genres and Category columns of the Google Play data set.

We'll build 2 functions that are found in the functions section to analyze our data:
1. Generate frequency tables that show percentages (freq_table)
2. Another function to display the percentages in descending order (display_table)

##### Apple Store prime_genre

In [84]:
display_table(AppleStore_Free,11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most common genre found in the Apple Store for Free English applications are resoundingly Gaming apps, consisting of 58% of the applications in this dataset. Entertainment is second, consisting of 7.88 % of the free English applications.

Whats important to note is that the App Store is dominated by Apps that are used for fun (Gaming, Entertainment, Photo&Video, Social Networking, Shopping, Sports, Music). Apps for practical use are more rare.

Another observation I notice is that genre's such as book, navigation, weather, and medical each have less than 1% of the number of free English applications in the App Store. From my own experience, I have found that these categories are often dominated by few apps that are used heavily (Google Maps, The Weather Channel, etc). Because these genre's have a low percentage of total free English apps does not mean they should be discounted as poorly performing genres. 

The fact that fun apps are most common does not imply that they have the greatest number of users.

Let's now examine the Category and Genre columns of the Google Play dataset (two columns that are similar)

##### Google Play Category

In [81]:
display_table(GooglePlay_Free,1) 

FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 0

##### Google Play Genres

In [83]:
display_table(GooglePlay_Free,9)

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.317161232088458
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051
S

The most common free English application genres in the Google Play store are Family (18.89%), Game (9.72%), and Tools (8.46%)

The layout of apps in the Google Play store is significantly different than the Apple Store at first glance. However, If you explore the Family category of the Google Play Store website at : https://play.google.com/store/apps/category/FAMILY, you find that majority of the applications categorized as "Family" are Games designed for children.

Although Gaming accounts for roughly 29% of the applications in the Google Play Store, productivity applications seem to be more common in the Google Play store than the App Store. 

Looking at the Genres column, we find that this section is much more granular than columns. Because it is so granular, we will focus on the Category column as we are looking at the big picture. 

We have found that the App Store is dominated by apps that are used for fun, while Google Play shows a better balance between for-fun and practical applcations.

Now we will explore what kind of applications have the most users.

# Most Popular Apps by Genre in the App Store

We will find out what genres are most popular by calculating the average number of installs for each app genre. The Google Play dataset has this information in the "Installs" column, but this info is missing for the App Store. We will use "rating_count_tot" for the App Store instead, where we will use the total number of user ratings as a proxy for installs.

Below we calculate the average number of user ratings per genre on the App Store

In [85]:
freq_table(AppleStore_Free,11)

{'Social Networking': 3.2898820608317814,
 'Photo & Video': 4.9658597144630665,
 'Games': 58.16263190564867,
 'Music': 2.0484171322160147,
 'Reference': 0.5586592178770949,
 'Health & Fitness': 2.0173805090006205,
 'Weather': 0.8690254500310366,
 'Utilities': 2.5139664804469275,
 'Travel': 1.2414649286157666,
 'Shopping': 2.60707635009311,
 'News': 1.3345747982619491,
 'Navigation': 0.186219739292365,
 'Lifestyle': 1.5828677839851024,
 'Entertainment': 7.883302296710118,
 'Food & Drink': 0.8069522036002483,
 'Sports': 2.1415270018621975,
 'Book': 0.4345127250155183,
 'Finance': 1.1173184357541899,
 'Education': 3.662321539416512,
 'Productivity': 1.7380509000620732,
 'Business': 0.5276225946617008,
 'Catalogs': 0.12414649286157665,
 'Medical': 0.186219739292365}

In [89]:
for genre in freq_table(AppleStore_Free,11):
    total = 0 
    len_genre = 0 
    
    for row in AppleStore_Free:
        genre_app = row[11]
        if genre_app == genre:
            num_Rating = float(row[5])
            total += num_Rating
            len_genre += 1 
    
    avg_users = (total/len_genre)
    
    print(genre,':',avg_users)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


In the App Store, Navigation apps get the most user ratings per app (86090 reviews/app). Reference apps are second (74,942 reviews/app), followed by Social Networking (71,548 reviews/app), Music (57,326 reviews/app), and Weather (52,279 reviews/app).

While Navigation apps have the highest number of reviews, this category is heavily influenced by Waze and Google Maps.
The same applies to Social Networking (Facebook, Pinterest, Skype, and Messenger dominate). Same with Music (Pandora, Spotify, Shazam).

The top genres in the app store by user ratings are dominated by few apps at the top of the category that heavily skew the results while the rest of the apps struggle to get about 10,000 reviews overall.

A good category that can provide opportunity could be those that has an equal distrubtion of reviews after the top results that are skewing the data, and applications that users spend alot of time engaging with the app and do not require setting up an entire business around the app. Some genre's that do not interest us include:

1. Weather - people do not spend alot of time in the weather app. People glance at the weather for less than a minute and close the app. Additionally, getting reliable live weather data would require us to connect to potentially non-free APIs or to set up our own equipment across the globe, so the app wouldn't be free.
2. Food and Drink - Examples are Coffee Apps or a Delivery service. We would have to establish kitchens and delivery drivers, so this is not feasible.

One potential category that could be an opportunity is the Reference category. Bible and Dictionary.com dominate this category, but this category consist of easy to create applications. We can take other holy scriptures such as the Bible and transform it into an app. We can provide easy features such as daily quotes, an audio version, quizzes, etc.

As we noted before, it seems the Apple Store is satured with gaming / for-fun apps. It will be hard to get an app in this category that will take off because there is so much competition. While there are not many practical apps, several practical categories are the highest performing genres, which could allow for us to be more competitive with our app.

In [95]:
for app in AppleStore_Free:
    if app[11] == "Navigation":
        print(app[1],":",app[5])

Kindle – Read eBooks, Magazines & Textbooks : 252076
Audible – audio books, original series & podcasts : 105274
Color Therapy Adult Coloring Book for Adults : 84062
OverDrive – Library eBooks and Audiobooks : 65450
HOOKED - Chat Stories : 47829
BookShout: Read eBooks & Track Your Reading Goals : 879
Dr. Seuss Treasury — 50 best kids books : 451
Green Riding Hood : 392
Weirdwood Manor : 197
MangaZERO - comic reader : 9
ikouhoushi : 0
MangaTiara - love comic reader : 0
謎解き : 0
謎解き2016 : 0


In [91]:
for app in AppleStore_Free:
    if app[11] == "Social Networking":
        print(app[1],":",app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

In [93]:
for app in AppleStore_Free:
    if app[11] == "Reference":
        print(app[1],":",app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In [92]:
for app in AppleStore_Free:
    if app[11] == "Music":
        print(app[1],":",app[5])

Pandora - Music & Radio : 1126879
Spotify Music : 878563
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
SoundCloud - Music & Audio : 135744
Magic Piano by Smule : 131695
Smule Sing! : 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Amazon Music : 106235
SoundHound Song Search & Music Player : 82602
Sonos Controller : 48905
Bandsintown Concerts : 30845
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
My Mixtapez Music : 26286
Sing Karaoke Songs Unlimited with StarMaker : 26227
Ringtones for iPhone & Ringtone Maker : 25403
Musi - Unlimited Music For YouTube : 25193
AutoRap by Smule : 18202
Spinrilla - Mixtapes For Free : 15053
Napster - Top Music & Radio : 14268
edjing Mix:DJ turntable to remix and scratch music : 13580
Free Music - MP3 Streamer & Playlist Manager Pro : 13443
Free Piano app by Yokee : 13016
Google Play Music : 10118
Certified Mixtapes - Hip Hop Albums & Mixtapes : 9975
TIDAL : 7398
YouTube Mu

In [94]:
for app in AppleStore_Free:
    if app[11] == "Weather":
        print(app[1],":",app[5])

The Weather Channel: Forecast, Radar & Alerts : 495626
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking : 208648
WeatherBug - Local Weather, Radar, Maps, Alerts : 188583
MyRadar NOAA Weather Radar Forecast : 150158
AccuWeather - Weather for Life : 144214
Yahoo Weather : 112603
Weather Underground: Custom Forecast & Local Radar : 49192
NOAA Weather Radar - Weather Forecast & HD Radar : 45696
Weather Live Free - Weather Forecast & Alerts : 35702
Storm Radar : 22792
QuakeFeed Earthquake Map, Alerts, and News : 6081
Moji Weather - Free Weather Forecast : 2333
Hurricane by American Red Cross : 1158
Forecast Bar : 375
Hurricane Tracker WESH 2 Orlando, Central Florida : 203
FEMA : 128
iWeather - World weather forecast : 80
Weather - Radar - Storm with Morecast App : 78
Yurekuru Call : 53
Weather & Radar : 37
WRAL Weather Alert : 25
Météo-France : 24
JaxReady : 22
Freddy the Frogcaster's Weather Station : 14
Almanac Long-Range Weather Forecast : 12
TodayAir

# Most Popular Apps by Genre on Google Play

For Google Play, we will use the Installs column. However, upon further inspection, the install numbers arent very precise and are open ended values (1000+,5000+, etc.). 

In [96]:
display_table(GooglePlay_Free,5)

1,000,000+ : 15.728308699086089
100,000+ : 11.55365000564143
10,000,000+ : 10.549475346947986
10,000+ : 10.199706645605326
1,000+ : 8.394448832223853
100+ : 6.916393997517771
5,000,000+ : 6.826131106848697
500,000+ : 5.562450637481666
50,000+ : 4.772650344127271
5,000+ : 4.513144533453684
10+ : 3.542818458761142
500+ : 3.2494640640866526
50,000,000+ : 2.3017037120613786
100,000,000+ : 2.1324607920568655
50+ : 1.9180864267178157
5+ : 0.7898002933543946
1+ : 0.5077287600135394
500,000,000+ : 0.270788672007221
1,000,000,000+ : 0.2256572266726842
0+ : 0.045131445334536835


As a workaround, we will leave the numbers the way they are ad consider an app with "100,000+" installs as having 100,000 installs, and so on.

First we will have to convert each install number from a string to a float, also removing commas and other symbols that will cause an error while converting to float.

In [97]:
freq_table(GooglePlay_Free,1)

{'ART_AND_DESIGN': 0.6431230960171499,
 'AUTO_AND_VEHICLES': 0.9251946293580051,
 'BEAUTY': 0.5979916506826132,
 'BOOKS_AND_REFERENCE': 2.1437436533904997,
 'BUSINESS': 4.592124562789123,
 'COMICS': 0.6205573733498815,
 'COMMUNICATION': 3.2381812027530184,
 'DATING': 1.8616721200496444,
 'EDUCATION': 1.1621347173643235,
 'ENTERTAINMENT': 0.9590432133589079,
 'EVENTS': 0.7108202640189552,
 'FINANCE': 3.7007785174320205,
 'FOOD_AND_DRINK': 1.241114746699763,
 'HEALTH_AND_FITNESS': 3.0802211440821394,
 'HOUSE_AND_HOME': 0.8236488773552973,
 'LIBRARIES_AND_DEMO': 0.9364774906916393,
 'LIFESTYLE': 3.9038700214374367,
 'GAME': 9.725826469592688,
 'FAMILY': 18.898792733837304,
 'MEDICAL': 3.5315355974275078,
 'SOCIAL': 2.6627552747376737,
 'SHOPPING': 2.245289405393208,
 'PHOTOGRAPHY': 2.944826808078529,
 'SPORTS': 3.396141261423897,
 'TRAVEL_AND_LOCAL': 2.335552296062281,
 'TOOLS': 8.462146000225657,
 'PERSONALIZATION': 3.317161232088458,
 'PRODUCTIVITY': 3.8925871601038025,
 'PARENTING': 0.

In [100]:
for category in freq_table(GooglePlay_Free,1):
    total = 0
    len_category = 0
    
    for row in GooglePlay_Free:
        category_app = row[1]
        if category_app == category:
            numInstalls = row[5]
            numInstalls = numInstalls.replace("+","")
            numInstalls = numInstalls.replace(",","")
            total += float(numInstalls)
            len_category+=1
    
    avg_installs = total/len_category
    
    print(category,":",avg_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3697848.1731343283
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On Average, Communication apps have the most installs (38,456,119). This number is heavily skewed by certain apps such as WhatsApp, Messenger, Skype, Gmail, etc) that have over a billion installs, while a few others have over 100 million.

In [106]:
for app in GooglePlay_Free:
    if app[1] == "COMMUNICATION" and (app[5] == "1,000,000,000+" or app[5] =='500,000,000+' or app[5] == '100,000,000+'):
        print(app[0],":",app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

What would the average number of installs be if we removed apps with over 100 million installs? The average would be reduced by nearly 10 times the number of installs. 

In [108]:
under_100mil = []

for app in GooglePlay_Free:
    numInstalls = app[5]
    numInstalls = numInstalls.replace("+","")
    numInstalls = numInstalls.replace(",","")
    if (app[1]=="COMMUNICATION") and (float(numInstalls)<100000000):
        under_100mil.append(float(numInstalls))

sum(under_100mil)/len(under_100mil)

3603485.3884615386

The same trend occurs in several other top categories, such as video players (dominated by Youtube, Google Play Movies, etc.), social media (Facebook, Instagram, Twitter), photography, and productivity apps (Word, Dropbox). It will be hard to compete with apps that dominate certain sectors, so it is not worthwile to develop an app in this category. The game genre is not dominated by certain apps as much as social media or communication, but as discussed previously this category is heavily saturated already.

The Books and Reference category stands out with Google Play as well. We found this could already be an opportunity in the app store, and it has an average number of installs of 8,767,811. Let's expore this genre more in depth

In [114]:
for app in GooglePlay_Free:
    if app[1] == "BOOKS_AND_REFERENCE":
        print(app[0],":",app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

This category has a variety of apps but also a small number of extremely popular apps that would skew the average significantly. Only 5 applications have over 100 million installs. This category shows opportunity.

In [115]:
for app in GooglePlay_Free:
    if app[1] == "BOOKS_AND_REFERENCE" and (app[5] == "1,000,000,000+" or app[5] =='500,000,000+' or app[5] == '100,000,000+'):
        print(app[0],":",app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


Let's explore applications in this category that are in the middle of the pack, between 1 million and 100 million downloads. In the middle of the pack, the apps that are most popular are readers, dictionaries, and translators. There are many of these apps, so we will stray away from developing one of these apps. There are a few apps built around holy scriptures like the Quran.

An opportunity that seems to exist for both the Apple Store and Google play is building an app around a popular book (holy or not). We will need to add additional features that will distinguish our app from apps that provide just the raw version of the book, as there are many of those. 

In [116]:
for app in GooglePlay_Free:
    if app[1] == "BOOKS_AND_REFERENCE" and (app[5] == "1,000,000+" or app[5] =='5,000,000+'
                                            or app[5] == '10,000,000+' or app[5] == '50,000,000+'):
        print(app[0],":",app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

# Conclusion

In this project, we analyzed data focused the Google Play and App Store applications that were free and intended for English speaking audiences. Our goal was to identify an app profile that seemed profitable in both marketplaces so we can develop an application that will get all of its revenue from ad views, which is dependent on the number of users viewing the add. 

We concluded that building a reference application could be the most profitable. Turning a popular book and turning it into an app that has additional features such as a built in dictionary, forum, quizzes, audio files could be a profitable venture in both marketplaces. Building an app in this category will require no additional resources to be built out (such as a delivery service), and people will spent large amounts of time in this app if they are reading an entire book. This provides an opportunity for alot of views for the advertisements on the app, which is our main source of revenue.